# Entity Matching with LLMs

In this notebook we present the pyJedAI-llm approach in the well-known ABT-BUY dataset. Clean-Clean ER in the link discovery/deduplication between two sets of entities. 

Dataset: __Abt-Buy dataset__ (D1)

The Abt-Buy dataset for entity resolution derives from the online retailers Abt.com and Buy.com. The dataset contains 1076 entities from abt.com and 1076 entities from buy.com as well as a gold standard (perfect mapping) with 1076 matching record pairs between the two data sources. The common attributes between the two data sources are: product name, product description and product price.

## Prerequisites 

### [Ollama](https://ollama.com)

Ollama is an open-source tool that allows you to run large language models (LLMs) directly on your local machine.

[Download](https://ollama.com/download) or: 


In [ ]:
#Linux Command 

!curl -fsSL https://ollama.com/install.sh | sh

Open new terminal and run `$ollama serve` or: 

In [ ]:
# start ollama server as a background process
import subprocess

process = subprocess.Popen("ollama serve", shell=True)

Error: listen tcp 127.0.0.1:11434: bind: address already in use


## How to install?

### pyJedAI

pyJedAI is an open-source library that can be installed from PyPI.

For more: [pypi.org/project/pyjedai/](https://pypi.org/project/pyjedai/)

In [5]:
!pip uninstall pyjedai -y

Found existing installation: pyjedai 0.2.5
Uninstalling pyjedai-0.2.5:
  Successfully uninstalled pyjedai-0.2.5


In [ ]:
!pip show pyjedai

Imports

In [1]:
import os
import sys
import pandas as pd
import networkx
import ollama
from networkx import draw, Graph

In [2]:
import pyjedai
from pyjedai.utils import (
    text_cleaning_method,
    print_clusters,
    print_blocks,
    print_candidate_pairs
)
from pyjedai.evaluation import Evaluation

# Data Reading

pyJedAI in order to perfrom needs only the tranformation of the initial data into a pandas DataFrame. Hence, pyJedAI can function in every structured or semi-structured data. In this case Abt-Buy dataset is provided as .csv files. 


In [3]:
from pyjedai.datamodel import Data
from pyjedai.evaluation import Evaluation

In [5]:
d1 = pd.read_csv("./../data/ccer/D2/abt.csv", sep='|', engine='python', na_filter=False)
d2 = pd.read_csv("./../data/ccer/D2/buy.csv", sep='|', engine='python', na_filter=False)
gt = pd.read_csv("./../data/ccer/D2/gt.csv", sep='|', engine='python')

data = Data(dataset_1=d1,
            id_column_name_1='id',
            dataset_2=d2,
            id_column_name_2='id',
            ground_truth=gt)

# Extracting Candidate Pairs

In this notebook the main purpose is to guide the user through the process of using pyjedai with ollama. We are currently using the KNN-Join Filtering with over 90% recall. You can choose any other pyJedAI filtering method.

In [6]:
from pyjedai.joins import TopKJoin

join = TopKJoin(K=5, metric='cosine', tokenization='qgrams_multiset', qgrams=5)
graph = join.fit(data)

/home/teris45/pyJedAI-Dev/src/pyjedai/joins.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
Top-K Join (cosine): 2152it [00:03, 582.11it/s]                           


In [7]:
from pyjedai.llm_matching import OllamaMatching

llm_matcher = OllamaMatching('llama3.2:1b')


Pulling model llama3.2:1b from ollama


In [8]:
pairs = llm_matcher.process(prediction=graph, data=data, create_examples=True, suffix='tf')


Embeddings-NN Block Building [sdistilroberta, faiss, cuda]: 100%|██████████| 2152/2152 [00:00<00:00, 15606.85it/s]


Created ollama model llama3.2:1b-tf


Ollama Matching [llama3.2:1b-tf]: 100%|██████████| 5449/5449 [18:56<00:00,  4.62it/s]

In [9]:
llm_matcher.export_to_df(pairs)

,id1,id2
0,134,0
1,134,8
2,134,9
3,134,10
4,1020,0
...,...,...
4081,473,1051
4082,1019,1057
4083,1056,1070
4084,1062,1073


In [10]:
ev = llm_matcher.evaluate(pairs, verbose=True)

***************************************************************************************************************************
                                         Method:  Ollama Matching
***************************************************************************************************************************
Method name: Ollama Matching
Parameters: 
	LLM: llama3.2:1b-tf
	Prompt: You are given two record descriptions and your task is to identify
if the records refer to the same entity or not.

You must answer with just one word:
True. if the records are referring to the same entity,
False. if the records are referring to a different entity.

Example 1
record 1: skagen premium steel slimline mesh womens watch - 233xsgg skagen premium steel slimline mesh womens watch - 233xsgg/ stainless steel mesh band/ elegant round case/ mother-of-pearl dial/ chrome indicators 110
record 2: 233xsgg - skagen  70
Answer: True.
Example 2
record 1: sony vaio lv series silver all-in-one desktop computer